<a href="https://colab.research.google.com/github/jmachima/Archaeology_News/blob/main/Archaeology_News_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification with content from archaeology.org/news (with permission from the editor-in-chief of Archaeology - A publication of the Archaeological Institute of America)

In [ ]:
import tensorflow as tf
import os

In [ ]:
# TPU will be used for text classification with BERT (Bidirectional Encoder Representations from Transformers).

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
!pip install -q -U tensorflow-text
!pip install -q -U tf-models-official

In [ ]:
pip install transformers

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text  # A dependency of the preprocessing model
#import tensorflow_addons as tfa
import numpy as np

tf.get_logger().setLevel('ERROR')

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import accuracy_score
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig

In [ ]:
#bert-base-cased
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', output_attention=True)
# 14 text categories for text classification
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-cased',output_attentions=True, num_labels=14)

In [ ]:
from google.colab import files

# upload ARCHAIC_TrainAndValidate.csv for training and validation
files.upload()

In [ ]:
data = pd.read_csv('ARCHAIC_TrainAndValidate.csv')

Class = set(data['category'])  # List of unique categories

CL = dict(zip(Class,list(range(14)))) # convert categories into numericals

rvCL = dict(zip(list(range(14)),Class)) # reverse lookup for numerical label using another dictionary

print(CL)

{'Modern': 0, 'Roman': 1, 'South&Southeast Asia': 2, 'Pre-Human': 3, 'Paleolithic': 4, 'Near East': 5, 'Neolithic': 6, 'Iron Age': 7, 'Egyptian': 8, 'Bronze Age': 9, 'Native American': 10, 'East Asia': 11, 'Pre-Columbian': 12, 'Medieval': 13}


In [ ]:
# Function for assigning numerical category to news text
#Also check if a news piece has a sequence length longer than 384
def col_label(Table): 
 
  Table['Label'] = 0 # want integers only
  long=[]

  for i in Table.index:
      
      Table.at[i,'Label'] = CL[Table.at[i,'category']]
      tokens=bert_tokenizer.tokenize(Table.at[i,'clean text'])
      if len(tokens) > 384:
        print('i=',i, len(tokens))
        long.append([i,len(tokens)])
      else:
        pass
  return Table, long

In [ ]:
# Add a 'Label' column to the data table
data, over_384 = col_label(data)

In [ ]:
# Assign input IDs and attention masks

def Encode(table):

  sentences=table['clean text']
  labels=table['Label']

  input_ids=[]
  attention_masks=[]

  for sent in sentences:
      bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =384,pad_to_max_length = True,return_attention_mask = True)
      input_ids.append(bert_inp['input_ids'])
      attention_masks.append(bert_inp['attention_mask'])

  input_ids=np.asarray(input_ids)
  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  return input_ids, attention_masks, labels

In [ ]:
# IDs and masks for news texts

input_ids, attention_masks, labels = Encode(data)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split input IDs, labels, and masks for training and validation sets

train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.25, random_state=73)
print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (708, 384) Val input shape (237, 384)
Train label shape (708,) Val label shape (237,)
Train attention mask shape (708, 384) Val attention mask shape (237, 384)


In [ ]:
# Training and validation
# With early stopping when criterion is met

with strategy.scope():

  Loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  Metric = tf.keras.metrics.SparseCategoricalAccuracy()
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
  Optimizer = tf.keras.optimizers.Adam(learning_rate=0.000009,epsilon=1e-08)
  bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
  bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=14)
  bert_model.compile(loss=Loss, metrics=Metric,optimizer=Optimizer)
  history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=50,validation_data=([val_inp,val_mask],val_label),callbacks=[callback])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/50
23/23 [==============================] - 133s 3s/step - loss: 2.6693 - sparse_categorical_accuracy: 0.0621 - val_loss: 2.6291 - val_sparse_categorical_accuracy: 0.0675
Epoch 2/50
23/23 [==============================] - 3s 153ms/step - loss: 2.6092 - sparse_categorical_accuracy: 0.1041 - val_loss: 2.5197 - val_sparse_categorical_accuracy: 0.2489
Epoch 3/50
23/23 [==============================] - 3s 153ms/step - loss: 2.4860 - sparse_categorical_accuracy: 0.2262 - val_loss: 2.3455 - val_sparse_categorical_accuracy: 0.2996
Epoch 4/50
23/23 [==============================] - 3s 153ms/step - loss: 2.2955 - sparse_categorical_accuracy: 0.3630 - val_loss: 2.0849 - val_sparse_categorical_accuracy: 0.5063
Epoch 5/50
23/23 [==============================] - 3s 152ms/step - loss: 2.0315 - sparse_categorical_accuracy: 0.5261 - val_loss: 1.7799 - val_sparse_categorical_accuracy: 0.7173
Epoch 6/50
23/23 [==============================] - 3s 154ms/step - loss: 1.7449 - sparse_categorical

In [ ]:
# Split texts (raw data) into train and validation sets so that validation set can be examined.

X_tr, X_ts, Y_tr, Y_ts=train_test_split(data['clean text'],data['Label'],test_size=0.25, random_state=73)

# Make a table for the validation set alone
val_set = pd.DataFrame(X_ts)
val_set['Label'] = Y_ts
y_true = list(Y_ts)

In [ ]:
#Obtain predicted categories from the trained BERT model

pred = bert_model([val_inp,val_mask])

In [ ]:
# Use argmax to get Label of predicted category
# Reverse look-up for category names

def rev_label(pred, X,y_true):

  predicted =[]
  pred_cat = []
  true_cat = []
  for j in range(len(X)):
    index= np.argmax(pred[0][j])
    predicted.append(int(index))
    cat = rvCL[index]
    true = rvCL[y_true[j]]
    pred_cat.append(cat)
    true_cat.append(true)
  return predicted, pred_cat, true_cat

In [ ]:
# predicted for predicted label
# pred_cat for predicted category name
# true_cat for actual category
predicted, pred_cat, true_cat = rev_label(pred, X_ts, y_true)

val_set['predicted'] = predicted
val_set['true_cat'] = true_cat
val_set['pred_cat'] = pred_cat

In [ ]:
print('Incorrectly predicted news pieces in total:',len(val_set[val_set['Label']!=val_set['predicted']]))
val_set[val_set['Label']!=val_set['predicted']]

Incorrectly predicted news pieces in total: 16


,clean text,Label,predicted,true_cat,pred_cat
473,New Thoughts on Indonesia’s Prehistoric Volcan...,4,3,Paleolithic,Pre-Human
349,New Thoughts on Turkey’s Oldest Temple Complex...,6,9,Neolithic,Bronze Age
501,Study Explores the Origins of Pottery in Asia....,4,6,Paleolithic,Neolithic
403,High-Altitude Roundhouse Dated in Scotland. GL...,9,6,Bronze Age,Neolithic
269,A Persian Military Outpost Identified in Israe...,7,8,Iron Age,Egyptian
258,DNA Study Tackles Caribbean Migration History....,12,6,Pre-Columbian,Neolithic
465,"Rare 5,000-Year-Old Sword Identified in Venice...",9,13,Bronze Age,Medieval
456,Stone Tools Recovered from Cave in West India....,4,2,Paleolithic,South&Southeast Asia
173,Artifacts Unearthed at Estate Site in Scotland...,0,13,Modern,Medieval
223,"Crusader-Era Battlefield Identified. TEL AVIV,...",5,13,Near East,Medieval


In [ ]:
#Upload 2 more files for test data sets
# ARCHAIC_test.csv and ARCHAIC_2labels.csv
files.upload()

In [ ]:
# Contains test data set which is somewhat more difficult than any validation set
# Test data set has only 1 label per news story
Easy_test = pd.read_csv('ARCHAIC_test.csv')

In [ ]:
# A place to assure there are no duplicates from the training set in the test sets
DUP = []
arch_testing = list(Easy_test['url'])
for j in range(len(data['url'])):
  if data.at[j,'url'] in arch_testing:
    DUP.append(data.at[j,'url'])
    print(j)
  else:
    pass

#If there is anything in DUP, it has to be removed from the test data set.
DUP

[]

In [ ]:
# Create a numerical label column for the Easy_test table
Easy_test, Easy_len=col_label(Easy_test)

# encode the texts in Easy_test
test_input_ids, test_attention_masks, test_labels = Encode(Easy_test)

In [ ]:
# Predicting news category on the test set
Easy_test_pred = bert_model([test_input_ids,test_attention_masks])

# Add predicted label and category columns to the Easy_test table
es_predicted, es_pred_cat, es_true_cat = rev_label(Easy_test_pred, Easy_test['clean text'], test_labels)

Easy_test['predicted'] = es_predicted
Easy_test['true_cat'] = es_true_cat
Easy_test['pred_cat'] = es_pred_cat

In [ ]:
print('Wrongly predicted news pieces from test set:',len(Easy_test[Easy_test['Label']!=Easy_test['predicted']]))
Easy_test[Easy_test['Label']!=Easy_test['predicted']][['clean text','category','pred_cat']]

In [ ]:
# Accuracy for the main test set (1 label)

ACCU = sklearn.metrics.accuracy_score(Easy_test['Label'], Easy_test['predicted'], normalize=True, sample_weight=None)
print('Easy_test Accuracy =',round((ACCU*100),2))

Easy_test Accuracy = 86.56


There are news stories which cannot be assigned to a single category as the content shows an overlap of 2 categories or more. Each news piece in the test data set below has been given 2 labels only for simplicity even though some of them are relevant to 3 categories. The news stories below are considered much more difficult to classify than the single-label test data set.

In [ ]:
# A different test data set with 2 classes assgined to each news story
Hard_test = pd.read_csv('ARCHAIC_2labels.csv')

# Assign the primary label first (not necessarily the dominant class)
Hard_test, Hard_over_384 = col_label(Hard_test)

In [ ]:
# Set up the numerical label column for the 2nd category
Hard_test['2nd_Label'] = 0 # want integers only
pair = []

for i in Hard_test.index:
      
  Hard_test.at[i,'2nd_Label'] = CL[Hard_test.at[i,'2nd category']]
  b = Hard_test.at[i,'2nd_Label']
  a = Hard_test.at[i,'Label']
  twin = [a,b]
  pair.append(twin)

# Add a column with a list of two numerical labels for each news piece
Hard_test['twin label'] = pair

In [ ]:
Hard_input_ids,Hard_attention_masks, Hard_labels = Encode(Hard_test)
Hard_test_pred = bert_model([Hard_input_ids,Hard_attention_masks])

In [ ]:
# Collect predicted twin labels using the argsort function. The last two positions in the argsort results give max. and 2nd highest probability values
hd_predicted =[]

for l in range(len(Hard_test)):
  twin_index= (np.argsort(Hard_test_pred[0][l]))[-2 :]
  hd_predicted.append(twin_index)

Hard_test['predicted'] = hd_predicted  

In [ ]:
# Rearrange the order of columns in the table for readability
Hard_test = Hard_test[['url','clean text','category','2nd category','Label','2nd_Label','twin label','predicted']]

In [ ]:
Hard_test[['clean text','category','2nd category','Label','2nd_Label','twin label','predicted']]

,clean text,category,2nd category,Label,2nd_Label,twin label,predicted
0,"Finds Spanning 4,000 Years Unearthed in Southe...",Neolithic,Iron Age,6,7,"[6, 7]","[9, 7]"
1,Genome Study Yields New Date for Arrival of TB...,Medieval,Neolithic,13,6,"[13, 6]","[13, 6]"
2,Early Christian Chalice Unearthed in Northern ...,Roman,Medieval,1,13,"[1, 13]","[13, 1]"
3,Multidisciplinary Study Tracks Spread of Rice ...,East Asia,South&Southeast Asia,11,2,"[11, 2]","[11, 2]"
4,Possible Ancient Longboat Recovered in Ireland...,Neolithic,Medieval,6,13,"[6, 13]","[9, 6]"
5,Three Well-Preserved Ancient Boats Unearthed i...,Roman,Medieval,1,13,"[1, 13]","[7, 1]"
6,Leather Balls Found in Ancient Graves in North...,Bronze Age,East Asia,9,11,"[9, 11]","[7, 11]"
7,Volunteers Spot New Sites in Aerial Images of ...,Roman,Medieval,1,13,"[1, 13]","[13, 1]"
8,"Snake Altar Unearthed in Turkey. ANTALYA, TURK...",Roman,Iron Age,1,7,"[1, 7]","[7, 2]"
9,Study Examines Food and Gender in Bronze Age C...,Bronze Age,East Asia,9,11,"[9, 11]","[8, 9]"


In [ ]:
# Dictionary reference
CL

{'Bronze Age': 9,
 'East Asia': 11,
 'Egyptian': 8,
 'Iron Age': 7,
 'Medieval': 13,
 'Modern': 0,
 'Native American': 10,
 'Near East': 5,
 'Neolithic': 6,
 'Paleolithic': 4,
 'Pre-Columbian': 12,
 'Pre-Human': 3,
 'Roman': 1,
 'South&Southeast Asia': 2}

In [ ]:
# Custom scoring - both labels correct - 1, only one label correct - 0.5

score = 0

for j in Hard_test.index:
  if (Hard_test.at[j,'predicted'])[0] in Hard_test.at[j,'twin label']:
    score += 0.5
  else:
    pass
  if (Hard_test.at[j,'predicted'])[1] in Hard_test.at[j,'twin label']:
    score += 0.5
  else:
    pass

print('two-label accuracy =', round(score/(len(Hard_test))*100,2))


two-label accuracy = 71.5
